In [4]:
import pandas as pd
from globals import BASE_DIR, available_datasets


In [5]:
print(available_datasets)

['snowcard']


In [6]:
# RQ1: this one is for the condensed table that only includes comparison to BPR

for dataset in available_datasets:
    df = pd.read_csv(f"{BASE_DIR}/{dataset}_dataset/evaluation_results_rq1.csv")
    user_group_order = ["LowPop", "MedPop", "HighPop", "All"]
    df["user_group"] = pd.Categorical(
        df["user_group"], categories=user_group_order, ordered=True
    )

    df_pivot = df.pivot(
        index=["user_group"], columns="method", values=["ndcg", "arp", "poplift"]
    )

    df_pivot = df_pivot.sort_values(by=["user_group"], axis=0, ascending=True)
    df_reset = df_pivot.reset_index()

    latex_table = df_reset.to_latex(
        escape=False,
        index=False,
        multicolumn=True,
        multirow=True,
        column_format="lccccccccccccccc",
        header=True,
    )

    latex_table = latex_table.replace("\\midrule", "\\hline")
    latex_table = latex_table.replace("\\bottomrule", "\\hline")
    latex_table = latex_table.replace("user_group", "group")
    latex_table = latex_table.replace("cp_min_js", "$ BPR + CP_\Im$")
    latex_table = latex_table.replace("cp", "BPR + $CP_H$")
    latex_table = latex_table.replace("%", "\%")

    latex_output = f"""
\\begin{{table*}}[]
\\centering
\\resizebox{{\\textwidth}}{{!}}{{%
{latex_table}
}}
\\caption{{{dataset.capitalize()} evaluation results}}
\\label{{tab:{dataset}_eval}}
\\end{{table*}}
"""

    # Save LaTeX file
    with open(f"{BASE_DIR}{dataset}_dataset/{dataset}_eval_rq1.tex", "w") as f:
        f.write(latex_output)

In [7]:
for dataset in available_datasets:
    df = pd.read_csv(f"{BASE_DIR}/{dataset}_dataset/evaluation_results_rq2.csv")

    df = df.loc[df["model"].isin(["BPR", "LORE", "USG"])]
    df = df.loc[df["method"] != "upd"]
    df.drop(columns=["js"], inplace=True)

    user_group_order = ["LowPop", "MedPop", "HighPop", "All"]
    df["user_group"] = pd.Categorical(
        df["user_group"], categories=user_group_order, ordered=True
    )

    float_columns = df.select_dtypes(include="float").columns
    df[float_columns] = df[float_columns].round(4)

    df_pivot = df.pivot(
        index=["model", "user_group"],
        columns="method",
        values=["ndcg", "arp", "poplift"],
    )
    df_pivot = df_pivot.sort_values(by=["model", "user_group"], ascending=True)

    df_reset = df_pivot.reset_index()

    latex_table = df_reset.to_latex(
        escape=False,
        index=False,
        multicolumn=True,
        multirow=True,
        column_format="llcccccccc",  
        header=True,
    )
    latex_table = latex_table.replace("\\midrule", "\\hline")
    latex_table = latex_table.replace("\\bottomrule", "\\hline")
    latex_table = latex_table.replace("user_group", "group")
    latex_table = latex_table.replace("cp_min_js", "CP_\Im")
    latex_table = latex_table.replace("cp", "CP_H")
    latex_table = latex_table.replace("baseline", "Base")
    latex_table = latex_table.replace("%", "\%")
    latex_table = latex_table.replace("Δ=", "")

    # Wrap the LaTeX table in the full structure
    latex_output = f"""
\\begin{{table*}}[]
\\centering
\\resizebox{{\\textwidth}}{{!}}{{%
{latex_table}
}}
\\caption{{{dataset.capitalize()} evaluation results}}
\\label{{tab:{dataset}_eval}}
\\end{{table*}}
"""

    # Save LaTeX file
    with open(f"{BASE_DIR}/{dataset}_dataset/{dataset}_eval_rq2.tex", "w") as f:
        f.write(latex_output)